# <center>Masakhane - Machine Translation for African Languages (Using JoeyNMT)</center>
## <leftalign> Author : Ari Ramkilowan</leftalign>
## <leftalign> Language Pair : English - Sepedi</leftalign>
## <leftalign> Corpus : JW300 </leftalign>

<hr>

## Install JoeyNMT

In [1]:
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (149/149), done.
remote: Compressing objects: 100% (104/104), done.
remote: Total 2333 (delta 98), reused 72 (delta 45), pack-reused 2184
Receiving objects: 100% (2333/2333), 2.64 MiB | 5.03 MiB/s, done.
Resolving deltas: 100% (1619/1619), done.
Processing /content/joeynmt
     |████████████████████████████████| 276kB 24.8MB/s 
     |████████████████████████████████| 307kB 67.8MB/s 
     |████████████████████████████████| 215kB 36.6MB/s 
     |████████████████████████████████| 51kB 7.6MB/s 
     |████████████████████████████████| 747kB 69.7MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
  Created wheel for joeynmt: filename=joeynmt-0.0.1-cp36-none-any.whl size=73467 sha256=5e896def4c67279e034c19c21caeb2ec5567599582c93bb52abe938e59ec6f3d
  Stored in directory: /tmp/pip-ephem-wheel-cache-sgho47kl/wheels/db/01/db/751cc9f3e7f6faec127c43644ba250a3ea7ad200594aeda70a
  Created wheel fo

## Mount Google Drive



In [2]:
# If running on Google Colab - mount google drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# TODO : access data on kaggle kernels

In [0]:
# TODO : Access data on paperspace

In [0]:
# TODo : Access data on GCP

## Set your source and target languages


In [0]:

import os
import numpy as np
import pandas as pd

source_language = "en"
target_language = "nso" 
lc = True  # If True, lowercase the data.
seed = 42  # Random seed for shuffling.
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted
# aggressive : more attention and more dropout
vocab_size=4000
corpus = "JW300"

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["trg"] = target_language
os.environ["tag"] = tag
os.environ["vocab_size"] = str(vocab_size)
os.environ["corpus"] = corpus

In [4]:
# This will save it to a folder in our gdrive instead!
# !mkdir -p "/content/drive/My Drive/masakhane/$src-$trg-$tag"
gdrive_path = f"/content/drive/My Drive/masakhane/{source_language}-{target_language}-{tag}/"
os.environ["gdrive_path"] = gdrive_path
! echo $gdrive_path

/content/drive/My Drive/masakhane/en-nso-baseline/


In [5]:
# create path to joeynmt executables scripts, configs etc

joey_path = f"/content/joeynmt"
os.environ["joey_path"] = joey_path
! ls $joey_path/configs

iwslt14_deen_bpe.yaml		   transformer_reverse.yaml
iwslt_deen_bahdanau.yaml	   transformer_small.yaml
iwslt_envi_luong.yaml		   transformer_wmt17_ende.yaml
iwslt_envi_xnmt.yaml		   transformer_wmt17_lven.yaml
reverse.yaml			   wmt_ende_best.yaml
small.yaml			   wmt_ende_default.yaml
transformer_copy.yaml		   wmt_lven_best.yaml
transformer_iwslt14_deen_bpe.yaml  wmt_lven_default.yaml


## Download the global test set.
 **(This changes from time to time, do this just to make sure you have the most recent version)**


In [0]:
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
  
! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.en
! mv test.en-$trg.en test.en

! wget https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-$trg.$trg 
! mv test.en-$trg.$trg test.$trg

--2019-11-03 20:36:39--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-any.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 277791 (271K) [text/plain]
Saving to: ‘test.en-any.en.1’

test.en-any.en.1    100%[===================>] 271.28K  --.-KB/s    in 0.05s   

2019-11-03 20:36:40 (5.35 MB/s) - ‘test.en-any.en.1’ saved [277791/277791]

--2019-11-03 20:36:42--  https://raw.githubusercontent.com/juliakreutzer/masakhane/master/jw300_utils/test/test.en-nso.en
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

In [0]:
# Read the test data to filter from train and dev splits.
# Store english portion in set for quick filtering checks.
en_test_sents = set()
filter_test_sents = "test.en-any.en"
j = 0
blanks=[]
with open(filter_test_sents) as f:
  for line in f:
    en_test_sents.add(line.strip())
    if len(line)<=1:
      blanks.append(j)
    j += 1
print('Loaded {} global test sentences to filter from the training/dev data.'.format(j))

FileNotFoundError: ignored

In [0]:
# filter test set

source_file = f"test.{source_language}"
target_file = f"test.{target_language}"

source = []
target = []

with open(source_file) as f:
  source = f.readlines()
            
with open(target_file) as f:
  target = f.readlines()

df = pd.DataFrame(zip(source, target), columns=['source_sentence', 'target_sentence'])

# remove trailing newline chars
df['source_sentence'] = df['source_sentence'].str.rstrip('" \n')
df['target_sentence'] = df['target_sentence'].str.rstrip('" \n')

# remove leading newline chars
df['source_sentence'] = df['source_sentence'].str.lstrip('"')
df['target_sentence'] = df['target_sentence'].str.lstrip('"')

# remove rows with really short sentences
df = df[~(df['source_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters
df = df[~(df['target_sentence'].str.len() <8)] # remove rows wher esource text len <8 characters

# save the filtered test set
df['source_sentence'].to_csv(f'{source_file}', index=False, header=False, doublequote=False)
df['target_sentence'].to_csv(f'{target_file}', index=False, header=False, doublequote=False)

In [0]:
# copy test sets to gdrive
! cp test.$src "$gdrive_path"
! cp test.$trg "$gdrive_path"
! cp test.$src-any.$src "$gdrive_path"

## Import prepared dataset

In [0]:
import pandas as pd
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [0]:
# This csv has extra columns added but no preprocessing done. all preprocessing should be captured in the NMT modelling notebook

input_file = f"{gdrive_path}/{source_language}-{target_language}-{corpus}-new.csv"
df = pd.read_csv(input_file)

In [8]:
df.head()

,source_sentence,target_sentence
0,They Rejoiced in Eastern Europe,Ba Ile Ba Thaba Ka Bohlabela Bja Yuropa
1,THIS past summer tens of thousands of people f...,SELEMONG se se se fetilego batho ba masome a d...
2,"The streets of beautiful Budapest , Prague , Z...","Ditarata tša Budapest e botse , Prague , Zagre..."
3,These identified them as lovers of godly freed...,Tše di be di ba hlaola e le barati ba tokologo...
4,"For the first time ever , conventions were fre...","Ka lekga la mathomo - thomo , dikopano di ile ..."


In [9]:
# How many samples
size = len(df)
print(f"\n {size} samples in original text")
  


 622966 samples in original text


## Preprocess input data

In [10]:
## Preprocessing - Step 1 : Drop NaNs

df_pp = df.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping all NaNs")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 613348 entries, 0 to 622965
Data columns (total 2 columns):
source_sentence    613348 non-null object
target_sentence    613348 non-null object
dtypes: object(2)
memory usage: 285.1 MB

 9618(1.54 %) samples removed by dropping all NaNs


In [11]:
## Preprocessing - Step 2a : Drop all duplicates in Source (en) text

df_pp = df_pp.drop_duplicates(subset='source_sentence')
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Source sentence duplicates")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 570881 entries, 0 to 622965
Data columns (total 2 columns):
source_sentence    570881 non-null object
target_sentence    570881 non-null object
dtypes: object(2)
memory usage: 302.1 MB

 42467(6.92 %) samples removed by dropping Source sentence duplicates


In [12]:
## Preprocessing - Step 2b : Drop all duplicates in Target (zu) text

df_pp = df_pp.drop_duplicates(subset='target_sentence')
df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping Target sentence duplicates")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567792 entries, 0 to 622965
Data columns (total 2 columns):
source_sentence    567792 non-null object
target_sentence    567792 non-null object
dtypes: object(2)
memory usage: 359.6 MB

 3089(0.54 %) samples removed by dropping Target sentence duplicates


In [0]:
##  Preprocessing - Step 3 : Remove all numeric entries

pattern = r"([0-9]*\.?[0-9]*)"  # catch integers and decimals
import re
r = re.compile(pattern)

In [14]:
%%time
##  Preprocessing - Step 3a : Remove all numeric entries - Source text

df_pp['source_sentence'] = df_pp['source_sentence'].str.replace(pattern,"")
df_pp['source_sentence'] = df_pp['source_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from source text")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567720 entries, 0 to 622965
Data columns (total 2 columns):
source_sentence    567720 non-null object
target_sentence    567720 non-null object
dtypes: object(2)
memory usage: 331.8 MB

 72(0.01 %) samples removed by dropping nummeric entries from source text
CPU times: user 9.93 s, sys: 62 ms, total: 9.99 s
Wall time: 10 s


In [15]:
%%time
##  Preprocessing - Step 3b : Remove all numeric entries - Target text

df_pp['target_sentence'] = df_pp['target_sentence'].str.replace(r,"")
df_pp['target_sentence'] = df_pp['target_sentence'].replace("",np.nan)

df_pp = df_pp.dropna()
df_pp.info(memory_usage='deep')
new_size = len(df_pp)

print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping nummeric entries from target text")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567699 entries, 0 to 622965
Data columns (total 2 columns):
source_sentence    567699 non-null object
target_sentence    567699 non-null object
dtypes: object(2)
memory usage: 271.9 MB

 21(0.00 %) samples removed by dropping nummeric entries from target text
CPU times: user 11.9 s, sys: 99.8 ms, total: 12 s
Wall time: 12 s


#### Preprocessing - Step 4 :Get length of sentences and then drop really short sentences


In [16]:
%%time
# add length columns


df_pp['source_ch_len'] = df_pp['source_sentence'].str.len()
df_pp['source_w_len'] = [len(text.split()) for text in df_pp['source_sentence']] 
df_pp['target_ch_len'] = df_pp['target_sentence'].str.len()
df_pp['target_w_len'] = [len(text.split()) for text in df_pp['target_sentence']] 
df_pp.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567699 entries, 0 to 622965
Data columns (total 6 columns):
source_sentence    567699 non-null object
target_sentence    567699 non-null object
source_ch_len      567699 non-null int64
source_w_len       567699 non-null int64
target_ch_len      567699 non-null int64
target_w_len       567699 non-null int64
dtypes: int64(4), object(2)
memory usage: 289.2 MB
CPU times: user 3.07 s, sys: 22.3 ms, total: 3.09 s
Wall time: 3.09 s


In [0]:
# # character len distrn - source text - 
# df_pp['source_ch_len'].value_counts().sort_index()

In [0]:
# # character len distrn - target text
# df_pp['target_ch_len'].value_counts().sort_index()

In [0]:
## how many rows with source text <=2chars and what do they look like ?

In [0]:
# # how many single character sentences from source ?
# f"{df_pp['source_ch_len'].value_counts()[1]} single character source sentences"

# df_pp[df_pp['source_ch_len']<=1]

In [0]:
# # how many 2-character sentences from source ?
# f"{df_pp['source_ch_len'].value_counts()[2]} 2-character source sentences"

# df_pp[df_pp['source_ch_len']==2]

In [17]:
##  Preprocessing - Step 4a :  drop everything where the ch_len <=2 in source text

df_pp = df_pp[~(df_pp['source_ch_len'] <=2) ]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with source sentences <= 2 characters")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567486 entries, 0 to 622965
Data columns (total 6 columns):
source_sentence    567486 non-null object
target_sentence    567486 non-null object
source_ch_len      567486 non-null int64
source_w_len       567486 non-null int64
target_ch_len      567486 non-null int64
target_w_len       567486 non-null int64
dtypes: int64(4), object(2)
memory usage: 289.2 MB

 213(0.04 %) samples removed by dropping rows with source sentences <= 2 characters


In [18]:
##  Preprocessing - Step 4b :  drop everything where the ch_len <=2 in target text

df_pp = df_pp[~(df_pp['target_ch_len'] <=2) ]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with target sentences <= 2 characters")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 567470 entries, 0 to 622965
Data columns (total 6 columns):
source_sentence    567470 non-null object
target_sentence    567470 non-null object
source_ch_len      567470 non-null int64
source_w_len       567470 non-null int64
target_ch_len      567470 non-null int64
target_w_len       567470 non-null int64
dtypes: int64(4), object(2)
memory usage: 289.2 MB

 16(0.00 %) samples removed by dropping rows with target sentences <= 2 characters


In [19]:
%%time
##  Preprocessing - Step 5 :  remove text from test set

with open(f"{gdrive_path}/test.en-any.en") as f:
    rows = f.readlines()
test_set_en = [row.strip() for row in rows]


df_pp = df_pp[~df_pp['source_sentence'].str.strip().isin(test_set_en)]

df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows from test set")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566404 entries, 0 to 622965
Data columns (total 6 columns):
source_sentence    566404 non-null object
target_sentence    566404 non-null object
source_ch_len      566404 non-null int64
source_w_len       566404 non-null int64
target_ch_len      566404 non-null int64
target_w_len       566404 non-null int64
dtypes: int64(4), object(2)
memory usage: 288.7 MB

 1066(0.19 %) samples removed by dropping rows from test set
CPU times: user 1.36 s, sys: 34 ms, total: 1.39 s
Wall time: 2.05 s


In [20]:
%%time
##  Preprocessing - Step 6 :  remove the extra "
df_pp['source_sentence'] = df_pp['source_sentence'].map(lambda x: x.lstrip('"').rstrip('"'))
df_pp['target_sentence'] = df_pp['target_sentence'].map(lambda x: x.lstrip('"').rstrip('"'))


df_pp.info(memory_usage='deep')
new_size = len(df_pp)
print(f"\n {size-new_size}({100*(size-new_size)/size :.2f} %) samples removed by dropping rows with extra quotes")
size = new_size

<class 'pandas.core.frame.DataFrame'>
Int64Index: 566404 entries, 0 to 622965
Data columns (total 6 columns):
source_sentence    566404 non-null object
target_sentence    566404 non-null object
source_ch_len      566404 non-null int64
source_w_len       566404 non-null int64
target_ch_len      566404 non-null int64
target_w_len       566404 non-null int64
dtypes: int64(4), object(2)
memory usage: 288.7 MB

 0(0.00 %) samples removed by dropping rows with extra quotes
CPU times: user 1.34 s, sys: 3.74 ms, total: 1.35 s
Wall time: 1.35 s


## create dev df 

In [21]:
df_dev = df_pp[['source_sentence', 'target_sentence']]
# Shuffle the data to remove bias in dev set selection.
seed=42
df_dev = df_dev.sample(frac=1, random_state=seed).reset_index(drop=True)
df_dev.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566404 entries, 0 to 566403
Data columns (total 2 columns):
source_sentence    566404 non-null object
target_sentence    566404 non-null object
dtypes: object(2)
memory usage: 8.6+ MB


## Create train and dev sets

In [0]:
%%time
# This section does the split between train/dev for the parallel corpora then saves them as separate files
# We use 1000 dev test and the given test set.

# Do the split between dev/train and create parallel corpora
num_dev_patterns = 1000

# Optional: lower case the corpora - this will make it easier to generalize, but without proper casing.
if lc:  # Julia: making lowercasing optional
    df_dev["source_sentence"] = df_dev["source_sentence"].str.lower()
    df_dev["target_sentence"] = df_dev["target_sentence"].str.lower()

# Julia: test sets are already generated
dev = df_dev.tail(num_dev_patterns) # Herman: Error in original
stripped = df_dev.drop(df_dev.tail(num_dev_patterns).index)

with open(f"{gdrive_path}/train."+source_language, "w") as src_file, open(f"{gdrive_path}/train."+target_language, "w") as trg_file:
  for index, row in stripped.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")
    
with open(f"{gdrive_path}/dev."+source_language, "w") as src_file, open(f"{gdrive_path}/dev."+target_language, "w") as trg_file:
  for index, row in dev.iterrows():
    src_file.write(row["source_sentence"]+"\n")
    trg_file.write(row["target_sentence"]+"\n")

CPU times: user 1min 26s, sys: 284 ms, total: 1min 26s
Wall time: 1min 27s


In [22]:
# Doublecheck the format below. There should be no extra quotation marks or weird characters.
! head "$gdrive_path/train.$src"
! echo "================================="
! head "$gdrive_path/dev.$src"
! echo "================================="
! head "$gdrive_path/test.$src" 

name some of god’s gifts that we can enjoy while awaiting future blessings 
mark states that soldiers “ would hit him on the head with a reed and spit upon him and , bending their knees [ in mockery ] , they would do obeisance to him  ”
instead , our faith will be strengthened if we rely on god , show love for him , and keep his commandments 
another person decides that all days are the same 
there were mice and cockroaches to keep us company at night 
my feelings of failure were such that it was often too depressing for me to read the life - story articles , which often recount extraordinary accomplishments of jehovah’s people 
nevertheless , jesus said those surprising words for good reason 
especially where children are involved , many parents prefer to use the services of a professional who is not adversarial 
why has the number of children adopted in britain dropped drastically during the last  years ?
how important it is to recognize that satan and his demons want us to feel that

In [23]:
! head "$gdrive_path/train.$trg"
! echo "================================="
! head "$gdrive_path/dev.$trg"
! echo "================================="
! head "$gdrive_path/test.$trg"

hlalosa dimpho tše dingwe tša modimo tšeo re ka di thabelago ge re dutše re letetše ditšhegofatšo tša nakong e tlago 
mareka o bolela gore bahlabani “ ba be ba mo itia hlogo ka lehlaka , ba mo tshwela ka mare gomme ba [ mo kwera ka go ] thinya matolo pele ga gagwe ba mo khunamela  ”
go e na le moo , tumelo ya rena e tla matlafala ge e ba re ithekga ka modimo , re bontšha gore re a mo rata e bile re boloka melao ya gagwe 
motho yo mongwe o phetha ka gore matšatši ka moka a a swana 
go be go e - na le magotlo le maphene a go re tloša bodutu bošego 
go ikwa ga - ka ke paletšwe gantši go be go ntira gore ke nyame kudu ge ke bala diphihlelo , tšeo gantši di bego di hlalosa dilo tše di makatšago tšeo di fihleletšwego ke batho ba jehofa 
lega go le bjalo , go na le lebaka le le kwagalago leo le dirilego gore jesu a bolele mantšu ao a makatšago 
batswadi ba bantši ba kgetha go diriša ditirelo tša setsebi seo se sa tšeego lehlakore , kudu - kudu moo bana ba akaretšwago 
ke ka baka la’ng palo ya

## Preprocessing the Data into Subword BPE Tokens

- One of the most powerful improvements for agglutinative languages (a feature of most Bantu languages) is using BPE tokenization [ (Sennrich, 2015) ](https://arxiv.org/abs/1508.07909).

- It was also shown that by optimizing the umber of BPE codes we significantly improve results for low-resourced languages [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021) [(Martinus, 2019)](https://arxiv.org/abs/1906.05685)

- Below we have the scripts for doing BPE tokenization of our data. We use 4000 tokens as recommended by [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021). You do not need to change anything. Simply running the below will be suitable. 

In [0]:
%%time
! subword-nmt learn-joint-bpe-and-vocab --input  "$gdrive_path"train.$src  "$gdrive_path"train.$trg -s $vocab_size -o  "$gdrive_path"bpe.codes.$vocab_size --write-vocabulary  "$gdrive_path"vocab.$src  "$gdrive_path"vocab.$trg

# Apply BPE splits to the train, development and test data.
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"train.$src > "$gdrive_path"train.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$trg < "$gdrive_path"train.$trg > "$gdrive_path"train.bpe.$trg

! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"dev.$src > "$gdrive_path"dev.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$trg < "$gdrive_path"dev.$trg > "$gdrive_path"dev.bpe.$trg

! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$src < "$gdrive_path"test.$src > "$gdrive_path"test.bpe.$src
! subword-nmt apply-bpe -c "$gdrive_path"bpe.codes.$vocab_size --vocabulary "$gdrive_path"vocab.$trg < "$gdrive_path"test.$trg > "$gdrive_path"test.bpe.$trg


CPU times: user 1.27 s, sys: 231 ms, total: 1.5 s
Wall time: 3min 27s


In [0]:
# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py "$gdrive_path"train.bpe."$src" "$gdrive_path"train.bpe."$trg" --output_path "$gdrive_path"vocab.txt

In [0]:

# Some output
! echo "BPE Sotho Sentences"
! tail -n 5 "$gdrive_path"test.bpe.$trg
! echo "==========================================================================="
! echo "Combined BPE Vocab"
! tail -n 10 "$gdrive_path"vocab.txt  # Herman

# !cp joeynmt/data/$src$tgt$vocab_size/vocab.txt "$gdrive_path"

BPE Sotho Sentences
M@@ a@@ fe@@ lelong ke ile ka tu@@ ma ka go se bote@@ ge .
"@@ K@@ a morago ga gore ke ithu@@ te therešo , ke ile ka gana go tšwela pele ke dira seo , gaešita le ge mošomo woo o be o le@@ fa gabotse .@@ "
"@@ K@@ e mohlala o mobotse go barwa ba ka ba babedi , e bile ke ne@@ ilwe di@@ tokelo ka phuthegong .@@ "
G@@ ona bjale ke tu@@ mile ka go botega gare ga ba@@ hlahlo@@ bi ba ma@@ kgetho le bao ke dirago kgwebo le bona . ”
"@@ R@@ u@@ the o ile a hudu@@ gela I@@ si@@ rae@@ le , moo a bego a ka rapela M@@ o@@ dimo wa therešo .@@ "
Combined BPE Vocab
swantšho
̀@@
ė
▲
ể@@
kots@@
̱@@
heber@@
̆
]@@


## Creating the JoeyNMT Config

JoeyNMT requires a yaml config. We provide a template below. We've also set a number of defaults with it, that you may play with!

- We used Transformer architecture 
- We set our dropout to reasonably high: 0.3 (recommended in  [(Sennrich, 2019)](https://www.aclweb.org/anthology/P19-1021))

Things worth playing with:
- The batch size (also recommended to change for low-resourced languages)
- The number of epochs (we've set it at 30 just so it runs in about an hour, for testing purposes)
- The decoder options (beam_size, alpha)
- Evaluation metrics (BLEU versus Crhf4)

In [24]:
name = '%s%s%s%s' % (source_language, target_language, str(vocab_size),tag)
name

'ennso4000baseline'

In [0]:
# Create this dir before we run for the first time so we store check points
# !mkdir -p "$gdrive_path/pretrained/$src$trg$vocab_size$tag/" # Herman

In [25]:
# This creates the config file for our JoeyNMT system. It might seem overwhelming so we've provided a couple of useful parameters you'll need to update
# (You can of course play with all the parameters if you'd like!)

name = '%s%s%s%s' % (source_language, target_language, str(vocab_size),tag)
gdrive_path = os.environ["gdrive_path"]

# Create the config
config = """
name: "{name}"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "{gdrive_path}train.bpe"
    dev:   "{gdrive_path}dev.bpe"
    test:  "{gdrive_path}test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "{gdrive_path}vocab.txt"
    trg_vocab: "{gdrive_path}vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "{gdrive_path}pretrained/{name}/best.ckpt" # if uncommented, load a pre-trained model from this checkpoint
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "plateau"           # TODO: try switching from plateau to Noam scheduling
    patience: 5                     # For plateau: decrease learning rate by decrease_factor if validation score has not improved for this many validation rounds.
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0003
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4096
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 10                     # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 1000          # TODO: Set to at least once per epoch.
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True               # TODO: Set to True if you want to overwrite possibly existing models. 
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 4             # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256   # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.3
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.4
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8              # TODO: Increase to 8 for larger data.
        embeddings:
            embedding_dim: 256    # TODO: Increase to 512 for larger data.
            scale: True
            dropout: 0.3
        # typically ff_size = 4 x hidden_size
        hidden_size: 256         # TODO: Increase to 512 for larger data.
        ff_size: 1024            # TODO: Increase to 2048 for larger data.
        dropout: 0.4
""".format(name=name,
           gdrive_path=os.environ["gdrive_path"],
           source_language=source_language,
           target_language=target_language
          )

with open("joeynmt/configs/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

3308

# Train the Model

This single line of joeynmt runs the training using the config we made above

In [26]:
! cd joeynmt/configs; ls
! cp joeynmt/configs/transformer_$src$trg$vocab_size$tag.yaml "$gdrive_path/"

iwslt14_deen_bpe.yaml		    transformer_reverse.yaml
iwslt_deen_bahdanau.yaml	    transformer_small.yaml
iwslt_envi_luong.yaml		    transformer_wmt17_ende.yaml
iwslt_envi_xnmt.yaml		    transformer_wmt17_lven.yaml
reverse.yaml			    wmt_ende_best.yaml
small.yaml			    wmt_ende_default.yaml
transformer_copy.yaml		    wmt_lven_best.yaml
transformer_ennso4000baseline.yaml  wmt_lven_default.yaml
transformer_iwslt14_deen_bpe.yaml


In [27]:
%%time
# Train the model
# You can press Ctrl-C to stop. And then run the next cell to save your checkpoints! 
! cd joeynmt; python3 -m joeynmt train configs/transformer_$src$trg$vocab_size$tag.yaml

2020-02-14 09:06:08,970 Hello! This is Joey-NMT.
2020-02-14 09:06:10,079 Total params: 12158720
2020-02-14 09:06:10,081 Trainable parameters: ['decoder.layer_norm.bias', 'decoder.layer_norm.weight', 'decoder.layers.0.dec_layer_norm.bias', 'decoder.layers.0.dec_layer_norm.weight', 'decoder.layers.0.feed_forward.layer_norm.bias', 'decoder.layers.0.feed_forward.layer_norm.weight', 'decoder.layers.0.feed_forward.pwff_layer.0.bias', 'decoder.layers.0.feed_forward.pwff_layer.0.weight', 'decoder.layers.0.feed_forward.pwff_layer.3.bias', 'decoder.layers.0.feed_forward.pwff_layer.3.weight', 'decoder.layers.0.src_trg_att.k_layer.bias', 'decoder.layers.0.src_trg_att.k_layer.weight', 'decoder.layers.0.src_trg_att.output_layer.bias', 'decoder.layers.0.src_trg_att.output_layer.weight', 'decoder.layers.0.src_trg_att.q_layer.bias', 'decoder.layers.0.src_trg_att.q_layer.weight', 'decoder.layers.0.src_trg_att.v_layer.bias', 'decoder.layers.0.src_trg_att.v_layer.weight', 'decoder.layers.0.trg_trg_att.k_l

In [0]:
# Copy the created models from the notebook storage to google drive for persistant storage 
# !cp  -r joeynmt/models/${src}${trg}${vocab_size}${tag}_transformer/* "$gdrive_path""pretrained/$src$trg$vocab_size$tag/"
!cp  joeynmt/models/${src}${trg}${vocab_size}${tag}_transformer/*.ckpt "$gdrive_path""pretrained/$src$trg$vocab_size$tag"

In [0]:
# copy across the config file
!cp  joeynmt/configs/transformer_${src}${trg}${vocab_size}${tag}.yaml "$gdrive_path"

In [35]:
!ls joeynmt/models

ennso4000baseline_transformer


In [38]:
# Test our model
# ! cd joeynmt; python3 -m joeynmt test "$gdrive_path""transformer_${src}${trg}${vocab_size}${tag}.yaml"
! cd joeynmt; python3 -m joeynmt test "$gdrive_path""pretrained/$src$trg$vocab_size$tag/config.yaml"

2020-02-14 09:31:35,938 Hello! This is Joey-NMT.
2020-02-14 09:32:14,148  dev bleu:  44.43 [Beam search decoding with beam size = 5 and alpha = 1.0]
2020-02-14 09:33:21,929 test bleu:  15.40 [Beam search decoding with beam size = 5 and alpha = 1.0]


In [0]:
# plot losses
# ! cd joeynmt; python3 scripts/plot_validations.py "$gdrive_path""pretrained/$src$trg$vocab_size$tag/" --plot_values bleu PPL  --output bleu2-ppl.png

In [0]:
# ! cat "$gdrive_path""pretrained/$src$trg$vocab_size$tag/config5_tok.yaml"

cat: '/content/drive/My Drive/masakhane/en-nso-baseline/pretrained/ennso4000baseline/config5_tok.yaml': No such file or directory


In [0]:
# Translate mode is mopre interactive but almsot the same as running in test mode
! cd joeynmt; python3 -m joeynmt translate "$gdrive_path""pretrained/$src$trg$vocab_size$tag/config_5sent.yaml"


Please enter a source sentence (pre-processed): 
 J@@ e@@ sus said : " Y@@ o@@ u must love your neighbo@@ r as yourself . 
JoeyNMT: ▶ esus o itše : seou se swanetše go rata moagišani wa gago bjalo ka ge o ithata

Please enter a source sentence (pre-processed): 
 J@@ e@@ sus said : " Y@@ o@@ u must love your neighbo@@ r as yourself . 
JoeyNMT: ▶ esus o itše : seou se swanetše go rata moagišani wa gago bjalo ka ge o ithata

Please enter a source sentence (pre-processed): 

Bye.


In [0]:
# ! cat "$gdrive_path""pretrained/$src$trg$vocab_size$tag/config_5tok.yaml"

In [0]:
# ! cat test.nso 
# ! echo "================================================="
# ! cat test.en

In [0]:
!# Output our validation accuracy
# /content/drive/My Drive/masakhane/en-nso-baseline/train.nso
# ! cat "$gdrive_path""pretrained/$src$trg$vocab_size$tag/validations.txt"